In [ ]:
## BIGQUERY
import polars as pl
from google.cloud import bigquery, bigquery_storage

PROJECT = 'dmecoyfin-250928192534125'
client = bigquery.Client(PROJECT)

job = client.query(query)
bqstorage_client = bigquery_storage.BigQueryReadClient()

query = 'SELECT * from c02_delta where foto_mes in (202105,202106,202107, 202108)'

job = client.query(query)

# Uso Storage API para traer Arrow más rápido
arrow_table = job.result().to_arrow(bqstorage_client=bqstorage_client)
df_pl = pl.from_arrow(arrow_table)
import src.config as config
import re
from google.cloud import bigquery
from src.loader import select_data_c02
from src.features import get_numeric_columns_pl, creation_lags, creation_deltas

data = select_data_c02([202102])
# Columnas a excluir
exclude_cols = ["numero_de_cliente", "foto_mes", "clase_binaria1", "clase_binaria2", "clase_peso"]
# Creo array con columnas numéricas
numeric_cols = get_numeric_columns_pl(data, exclude_cols=exclude_cols)

# Creo tabla con lags
creation_lags(numeric_cols, config.NUN_WINDOW_LOAD)

# Creo tabla con deltas
creation_deltas(numeric_cols, config.NUN_WINDOW_LOAD)


def generate_average_delta_query(project_id, dataset_id, table_id, partition_date_str):
    """
    Genera una consulta SQL para calcular el promedio de todas las columnas
    que terminan en _delta_1 a _delta_5 para una fecha de partición específica.
    """
    client = bigquery.Client(project=project_id)
    table_path = f"{project_id}.{dataset_id}.{table_id}"

    # 1. Obtener todos los nombres de columnas de la tabla
    table = client.get_table(table_path)
    all_columns = [field.name for field in table.schema]

    # 2. Identificar las columnas base (ej. 'campo_a', 'otro_campo')
    #    Asumimos que todas las columnas que terminan en _delta_X tienen un prefijo común.
    #    Filtramos solo las columnas que tienen el patrón de delta
    delta_columns = [col for col in all_columns if re.search(r'_delta_[1-5]$', col)]

    # 3. Agrupar las columnas por su número de delta
    deltas_by_level = {i: [] for i in range(1, 6)}
    for col in delta_columns:
        # Extraer el nivel del delta (1, 2, 3, 4 o 5) del final del nombre de la columna
        match = re.search(r'_delta_([1-5])$', col)
        if match:
            level = int(match.group(1))
            deltas_by_level[level].append(col)

    # 4. Generar las cláusulas AVG para la consulta SQL
    avg_expressions = []
    for level in sorted(deltas_by_level.keys()):
        cols_at_level = deltas_by_level[level]
        if cols_at_level:
            # Calcular el promedio de las columnas en este nivel de delta
            # La sintaxis de BigQuery para promediar múltiples columnas es compleja,
            # así que usamos la media de los promedios individuales o pivotamos.
            # La mejor forma es calcular el promedio de cada columna por separado.

            # Lista de AVG individuales: AVG(col_1_delta_5), AVG(col_2_delta_5), ...
            individual_avgs = [f"AVG(`{col}`)" for col in cols_at_level]

            # Combinamos todos los promedios individuales en un solo cálculo de media global para ese nivel de delta
            # Esto es lo que solicitó el usuario: "para cada campo con delta_5 calcule su promedio"

            for col in cols_at_level:
                avg_expressions.append(f"AVG(`{col}`) AS `{col}_avg`")

    # 5. Ensamblar la consulta SQL final
    #    Asumimos que la tabla está particionada por una columna llamada 'fecha_particion'
    #    o similar, y que el valor es '202108'.
    query = f"""
    CREATE TABLE `{config.BQ_PROJECT}.{config.BQ_DATASET}.{table_id}_avg_delta_202108` AS
{",\n".join(["      " + expr for expr in avg_expressions])}
    FROM
        `{table_path}`
    WHERE
foto_mes = 202108    """

    return query


# --- Uso del script ---

# Reemplaza estos valores con los detalles de tu proyecto, conjunto de datos y tabla
PROJECT_ID = config.BQ_PROJECT
DATASET_ID = config.BQ_DATASET
TABLE_ID = "c02_delta"
PARTITION_MONTH = "202108"  # El mes que quieres analizar

try:
    sql_query = generate_average_delta_query(PROJECT_ID, DATASET_ID, TABLE_ID, PARTITION_MONTH)
    print("--- Consulta SQL Generada ---")
    print(sql_query)
    print("-----------------------------")

    # Opcional: Ejecutar la consulta directamente en BigQuery (descomentar para usar)
    client = bigquery.Client(project=config.BQ_PROJECT)
    # print("Ejecutando consulta en BigQuery...")
    query_job = client.query(sql_query)
    results = query_job.result()
    # print("Resultados:")
    # for row in results:
    #     print(row)

except Exception as e:
    print(
        f"\nOcurrió un error al intentar obtener el esquema de la tabla. Asegúrate de que las credenciales y los nombres son correctos. Error: {e}")


def generate_average_delta_query_pivot(project_id, dataset_id, table_id, partition_date_str):
    """
    Genera una consulta SQL para calcular el promedio de todas las columnas
    que terminan en _delta_1 a _delta_5, y pivota el resultado.
    """
    # Usaremos un cliente simulado si no estás en un entorno con credenciales activas
    # client = bigquery.Client(project=project_id)
    client = bigquery.Client(project=project_id)
    table_path = f"{project_id}.{dataset_id}.{table_id}"

    # 1. Obtener todos los nombres de columnas de la tabla
    table = client.get_table(table_path)
    all_columns = [field.name for field in table.schema]

    # 2. Identificar las columnas base (ej. 'campo_a', 'otro_campo')
    #    Asumimos que todas las columnas que terminan en _delta_X tienen un prefijo común.
    #    Filtramos solo las columnas que tienen el patrón de delta
    delta_columns = [col for col in all_columns if re.search(r'_delta_[1-5]$',
                                                             col)]  # --------------------------------------------------------------------------------------------------

    # 1. Identificar las columnas delta y sus bases
    delta_columns = [col for col in all_columns if re.search(r'_delta_[1-5]$', col)]

    # Diccionario para mapear nombre base a una lista de sus deltas [d1, d2, d3, d4, d5]
    base_fields = {}
    for col in delta_columns:
        # Usamos regex para separar el nombre base y el sufijo delta
        match = re.search(r'^(.*)_delta_([1-5])$', col)
        if match:
            base_name = match.group(1)
            delta_level = int(match.group(2))
            if base_name not in base_fields:
                base_fields[base_name] = [None] * 5
            base_fields[base_name][delta_level - 1] = col

    # 2. Generar la subconsulta que calcula los promedios globales por columna
    # Es importante mantener los nombres de columna originales para el UNPIVOT
    avg_expressions = [f"AVG(`{col}`) as `{col}`" for col in delta_columns]
    avg_select_list = ",\n".join(["      " + expr for expr in avg_expressions])

    subquery_alias = "GlobalAvgs"
    partition_date_str = "202108"  # Usamos la fecha que tenías en la query

    # 3. Generar la consulta SQL final utilizando UNPIVOT y PIVOT
    query = f"""
    CREATE OR REPLACE TABLE `{config.BQ_PROJECT}.{config.BQ_DATASET}.deltas_202108_zscore` AS
    WITH {subquery_alias} AS (
        SELECT
{avg_select_list}
        FROM
            `{table_path}`
        WHERE
            foto_mes = {partition_date_str}
    ),
    UnpivotedAvgs AS (
        -- Despivota los resultados de los promedios globales
        SELECT
            REPLACE(REGEXP_EXTRACT(col_name, r'^(.*)_delta_[0-9]+$'), '`', '') AS field_name_base,
            CAST(REGEXP_EXTRACT(col_name, r'_delta_([0-9]+)$') AS INT64) AS delta_level,
            avg_value
        FROM
            {subquery_alias}
        UNPIVOT(avg_value FOR col_name IN ({", ".join([f"`{col}`" for col in delta_columns])}))
    ),
    StandardizedAvgs AS (
        -- Aplica la estandarización Z-score a los promedios
        SELECT
            field_name_base,
            delta_level,
            avg_value,
            -- Calcula (Valor - Media) / Desviación Estándar para cada nivel de delta
            (avg_value - AVG(avg_value) OVER (PARTITION BY delta_level)) / STDDEV(avg_value) OVER (PARTITION BY delta_level) AS z_score
        FROM
            UnpivotedAvgs
    )
    -- Pivota los Z-scores para obtener el formato final deseado
    SELECT
        field_name_base AS nombre_campo,
        AVG(IF(delta_level = 1, z_score, NULL)) AS delta_1,
        AVG(IF(delta_level = 2, z_score, NULL)) AS delta_2,
        AVG(IF(delta_level = 3, z_score, NULL)) AS delta_3,
        AVG(IF(delta_level = 4, z_score, NULL)) AS delta_4,
        AVG(IF(delta_level = 5, z_score, NULL)) AS delta_5
    FROM
        StandardizedAvgs
    GROUP BY
        nombre_campo
    """

    return query


try:
    sql_query = generate_average_delta_query_pivot(PROJECT_ID, DATASET_ID, TABLE_ID, PARTITION_MONTH)
    print("--- Consulta SQL Generada ---")
    print(sql_query)
    print("-----------------------------")

    # Opcional: Ejecutar la consulta directamente en BigQuery (descomentar para usar)
    client = bigquery.Client(project=config.BQ_PROJECT)
    # print("Ejecutando consulta en BigQuery...")
    query_job = client.query(sql_query)
    results = query_job.result()
    # print("Resultados:")
    # for row in results:
    #     print(row)

except Exception as e:
    print(
        f"\nOcurrió un error al intentar obtener el esquema de la tabla. Asegúrate de que las credenciales y los nombres son correctos. Error: {e}")


## DELTAS PARA VARIOS FOTO MES
def generate_average_delta_query_multi_month(project_id, dataset_id, table_id):
    """
    Genera una consulta SQL para calcular y estandarizar los promedios de delta para dos meses,
    manteniendo el foto_mes en la tabla final.
    """
    client = bigquery.Client(project=project_id)
    table_path = f"{project_id}.{dataset_id}.{table_id}"

    # 1. Obtener todos los nombres de columnas de la tabla
    table = client.get_table(table_path)
    all_columns = [field.name for field in table.schema]
    # --------------------------------------------------------------------------------------------------

    # 1. Identificar las columnas delta y sus bases
    delta_columns = [col for col in all_columns if re.search(r'_delta_[1-5]$', col)]
    # Lista de nombres de columnas sin comillas invertidas para el UNPIVOT
    delta_columns_unquoted = [f"`{col}`" for col in delta_columns]

    # 2. Generar la consulta SQL final
    query = f"""
    CREATE TABLE `{config.BQ_PROJECT}.{config.BQ_DATASET}.deltas_multi_meses_zscore` AS
    WITH MonthlyAvgs AS (
        -- Calcula los promedios por mes y por columna delta
        SELECT
            foto_mes,
            {",\n      ".join([f"AVG(`{col}`) as `{col}`" for col in delta_columns])}
        FROM
            `{table_path}`
        WHERE
            foto_mes IN (202105,202106,202107, 202108)
        GROUP BY
            foto_mes
    ),
    UnpivotedMonthlyAvgs AS (
        -- Despivota los resultados de los promedios mensuales
        SELECT
            foto_mes,
            -- Extrae el nombre base (ej. 'edad') como 'feature'
            REPLACE(REGEXP_EXTRACT(col_name, r'^(.*)_delta_[0-9]+$'), '`', '') AS feature,
            -- Extrae el nivel del delta (ej. 1)
            CAST(REGEXP_EXTRACT(col_name, r'_delta_([0-9]+)$') AS INT64) AS delta_level,
            avg_value
        FROM
            MonthlyAvgs
        UNPIVOT(avg_value FOR col_name IN ({", ".join(delta_columns_unquoted)}))
    ),
    StandardizedAvgs AS (
        -- Aplica la estandarización Z-score.
        -- PARTITION BY delta_level asegura que estandarizamos 'delta_1' contra todos los 'delta_1'
        SELECT
            foto_mes,
            feature,
            delta_level,
            avg_value,
            (avg_value - AVG(avg_value) OVER (PARTITION BY delta_level)) / NULLIF(STDDEV(avg_value) OVER (PARTITION BY delta_level), 0) AS z_score
        FROM
            UnpivotedMonthlyAvgs
    )
    -- Pivota los Z-scores para obtener el formato final deseado
    SELECT
        foto_mes,
        feature,
        AVG(IF(delta_level = 1, z_score, NULL)) AS delta_1,
        AVG(IF(delta_level = 2, z_score, NULL)) AS delta_2,
        AVG(IF(delta_level = 3, z_score, NULL)) AS delta_3,
        AVG(IF(delta_level = 4, z_score, NULL)) AS delta_4,
        AVG(IF(delta_level = 5, z_score, NULL)) AS delta_5
    FROM
        StandardizedAvgs
    GROUP BY
        foto_mes, feature
    ORDER BY
        foto_mes, feature
    """

    return query


try:
    sql_query = generate_average_delta_query_multi_month(PROJECT_ID, DATASET_ID, TABLE_ID)
    print("--- Consulta SQL Generada ---")
    print(sql_query)
    print("-----------------------------")

    # Opcional: Ejecutar la consulta directamente en BigQuery (descomentar para usar)
    client = bigquery.Client(project=config.BQ_PROJECT)
    # print("Ejecutando consulta en BigQuery...")
    query_job = client.query(sql_query)
    results = query_job.result()
    # print("Resultados:")
    # for row in results:
    #     print(row)

except Exception as e:
    print(
        f"\nOcurrió un error al intentar obtener el esquema de la tabla. Asegúrate de que las credenciales y los nombres son correctos. Error: {e}")
